# Can you find the fish in state names?

This is my solution to the Riddler Classic from May 22nd, 2020

https://fivethirtyeight.com/features/somethings-fishy-in-the-state-of-the-riddler/

From Mark Bradwin comes a fishy puzzle about state names:

Ohio is the only state whose name doesn’t share any letters with the word “mackerel.” It’s strange, but it’s true.

But that isn’t the only pairing of a state and a word you can say that about — it’s not even the only fish! Kentucky has “goldfish” to itself, Montana has “jellyfish” and Delaware has “monkfish,” just to name a few.

What is the longest “mackerel?” That is, what is the longest word that doesn’t share any letters with exactly one state? (If multiple “mackerels” are tied for being the longest, can you find them all?)

Extra credit: Which state has the most “mackerels?” That is, which state has the most words for which it is the only state without any letters in common with those words?

For both the Riddler and the extra credit, please refer to Friend of the Riddler™ Peter Norvig’s word list:

https://norvig.com/ngrams/word.list

In [1]:
# import pandas and set display options
import pandas as pd
pd.set_option('display.max_columns', None)

# declare filepaths
word_path = r"C:\Users\************************\ANALYSIS\538 riddles\- data\wordlist.csv"
state_path = r"C:\Users\***********************\- ANALYSIS\538 riddles\- data\US states.csv"

# read in the data to dataframes, and convert all letters to lower case
wordlist = pd.read_csv(word_path, header=None, names =['word'], keep_default_na=False)
wordlist['word'] = wordlist['word'].str.lower()
stateslist = pd.read_csv(state_path, header=None, names = ['state'])
stateslist['state'] = stateslist['state'].str.lower()

# display head of each dataframe
display(wordlist.head())
display(stateslist.head())

,word
0,aa
1,aah
2,aahed
3,aahing
4,aahs


,state
0,alabama
1,alaska
2,arizona
3,arkansas
4,california


In [2]:
# initialise a list to store dictionaries of the letters of each word
letters_in_words = []

# loop through the wordlist, create a dictionarty of the letters of each word, and append to letters_in_words
for word in wordlist['word']:
    letter_dict = dict.fromkeys(word, True)  
    letters_in_words.append(letter_dict)

# create a dataframe to store the results    
df = pd.DataFrame(letters_in_words)
df.set_index(wordlist['word'], inplace=True)

# display the head and tail of the dataframe
display(df.head())
display(df.tail())

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
word,,,,,,,,,,,,,,,,,,,,,,,,,,
aa,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aah,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aahed,True,NaN,NaN,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aahing,True,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aahs,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
word,,,,,,,,,,,,,,,,,,,,,,,,,,
zythum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,True,True
zythums,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,NaN,True,True
zyzzyva,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,True,True
zyzzyvas,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,True
zzz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [3]:
# create a dataframe to store the results
initial_results = pd.DataFrame(columns=['state', 'word', 'word_length'])

# loop through the states, and create a list of unique letters in that state, removing whitespaces
for state in stateslist['state']:
    state_letters = []
    state_letters = list(set(state))
    state_letters.sort()
    if " " in state_letters: state_letters.remove(" ")  
    
    # create a temp df and filter columns for only the letters that the state contains
    # then filter the dataframe for rows (words) that are all NaN, i.e. they don't share any letters with that state
    temp_df = df[state_letters].copy()
    state_in_question = []
    state_in_question = list(temp_df.index[temp_df.isnull().all(1)])
    state_in_question = pd.DataFrame(data=state_in_question, columns = ['word'])
    state_in_question['state'] = state
    state_in_question['word_length'] = state_in_question['word'].str.len()
    
    # append the values in this dataframe to the final_results dataframe
    initial_results = initial_results.append(state_in_question, sort=False, ignore_index=True)

In [4]:
# sort values in final_results by state and then word_length
initial_results.sort_values(by = ['state', 'word_length'], ascending = False, inplace = True)

# create a final_results dataframe with duplicate words removed
final_results = initial_results.drop_duplicates(subset = 'word', keep = False)

# what are the top twenty largest mackerels
display(final_results.sort_values(by = 'word_length', ascending = False).head(20))

,state,word,word_length
3520,alabama,counterproductivenesses,23
268792,mississippi,hydrochlorofluorocarbon,23
3519,alabama,counterproductiveness,21
46431,alabama,unconscientiousnesses,21
42119,alabama,supposititiousnesses,20
46555,alabama,underconsciousnesses,20
133240,hawaii,overscrupulousnesses,20
370344,ohio,transcendentalnesses,20
373218,ohio,untranslatablenesses,20
441845,utah,incompressiblenesses,20


In [5]:
# display the number of 'mackerels' for each state 
for state in stateslist['state']:
    no_of_mackerels = final_results[final_results['state'] == state].shape[0]
    display(state + " has " + str(no_of_mackerels) + " mackerels")

'alabama has 8274 mackerels'

'alaska has 1261 mackerels'

'arizona has 0 mackerels'

'arkansas has 0 mackerels'

'california has 0 mackerels'

...

'virginia has 107 mackerels'

'washington has 0 mackerels'

'west virginia has 0 mackerels'

'wisconsin has 60 mackerels'

'wyoming has 1364 mackerels'